In [3]:
import os
import pandas as pd
import json

def parse_data(filename):
    df1 = pd.read_csv(filename)
    df2 = df1.rename(columns={"trnOpDate":"乘車日",
            "staCode":"車站代碼",
            "gateInComingCnt":"進站人數",
            "gateOutGoingCnt":"出站人數"})
    with open('車站基本資料集.json',encoding='utf-8') as file:
        json_data = json.load(file)
        stations_info = pd.DataFrame(json_data,columns=['stationCode','stationName'])
        stations_info = stations_info.astype({'stationCode':'int32'})
    df3 = pd.merge(df2,stations_info,left_on='車站代碼',right_on='stationCode')
    df4 = df3.rename(columns={'stationName':'車站名稱'})
    df5 = df4.reindex(columns=['乘車日','車站名稱','進站人數','出站人數'])
    df5['乘車日'] = pd.to_datetime(df5['乘車日'].astype(str))
    return df5

def main():
    current_dir = os.getcwd()
    csv_dir = os.path.join(current_dir,'每日各站進出站人數')
    files_and_dirs = os.listdir(csv_dir)
    all_csv_files = [filename for filename in files_and_dirs if '每日各站進出站人數' in filename]
    sorted_csv_files = sorted(all_csv_files)
    all_final_dfs = [] 
    for filename in sorted_csv_files:
        abs_filename = os.path.join(csv_dir,filename)
        all_final_dfs.append(parse_data(abs_filename))
    df_done = pd.concat(all_final_dfs)
    df_done1 = df_done.set_index('乘車日')
    return df_done1 


if __name__ == '__main__':
    df_done1 = main()


In [6]:
df_done2 = df_done1.query('車站名稱 in ("台北","臺中","高雄")')

In [7]:
df_done2

,車站名稱,進站人數,出站人數
乘車日,,,
2019-04-23,臺中,18885,18430
2019-04-23,高雄,13281,12606
2019-04-24,臺中,18773,18509
2019-04-24,高雄,13549,13333
2019-04-25,臺中,18894,18055
...,...,...,...
2023-12-29,高雄,17733,18580
2023-12-30,臺中,42661,43502
2023-12-30,高雄,18901,19072


In [10]:
years = df_done2.index.to_series().dt.year.values
months = df_done2.index.to_series().dt.month.values

In [11]:
groupby=df_done2.groupby(by=[years,months,'車站名稱'])
df2 = groupby[['進站人數','出站人數']].sum()
df2

進站人數    出站人數
        車站名稱                
2019 4  臺中    196819  196322
        高雄    131501  129092
     5  臺中    756624  753259
        高雄    496946  501062
     6  臺中    806261  811109
...              ...     ...
2023 10 高雄    448739  450459
     11 臺中    748596  735726
        高雄    432731  434283
     12 臺中    861473  854234
        高雄    480623  485788

[114 rows x 2 columns]

In [12]:
df2.unstack(level='車站名稱')

進站人數            出站人數        
車站名稱         臺中      高雄      臺中      高雄
2019 4   196819  131501  196322  129092
     5   756624  496946  753259  501062
     6   806261  516769  811109  515675
     7   813097  544617  741690  548130
     8   826856  528537  770365  528947
     9   811352  513354  816947  510600
     10  821320  532588  821740  534422
     11  798980  511155  799602  512241
     12  865393  538406  870876  542660
2020 1   853257  532721  851720  532198
     2   662240  391059  674267  388085
     3   566732  349488  563303  350412
     4   473254  282979  474284  281759
     5   582403  332425  583891  332177
     6   667794  403193  672598  404317
     7   745164  455539  755891  454959
     8   780932  477900  791989  475392
     9   680812  420352  683041  421117
     10  777955  465307  789753  468930
     11  725054  433081  734713  433672
     12  759804  458309  765387  461877
2021 1   679954  417812  684093  417228
     2   699411  420671  709872  424129
     3   689972  423728  695948  427297
     4   707678  425802  709312  427424
     5   400709  253633  403865  254419
     6   100667   78896  101790   78602
     7   156347  110182  158890  109769
     8   335429  204229  340135  206019
     9   444338  266194  444401  268339
     10  582067  340491  583004  344892
     11  590041  352573  590804  357140
     12  681566  393457  680485  403794
2022 1   581566  324785  578734  323423
     2   548708  300946  559293  306106
     3   589879  343998  586534  348242
     4   524714  302446  525196  303592
     5   345864  213052  342128  214884
     6   380827  218836  382892  222175
     7   569848  317279  573803  320510
     8   637010  367222  641151  368141
     9   604952  339369  606791  343951
     10  658218  366792  663179  370025
     11  628440  351005  628654  354034
     12  716617  384875  713199  393045
2023 1   780852  411788  780312  412376
     2   703190  380023  705011  380976
     3   676481  385126  667278  392025
     4   760188  421624  764348  420199
     5   707246  385261  702431  386588
     6   711998  386020  707219  388091
     7   763165  400562  764193  397492
     8   778790  422659  779324  421531
     9   742373  394908  729035  394728
     10  810733  448739  811878  450459
     11  748596  432731  735726  434283
     12  861473  480623  854234  485788